In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
train_df = pd.read_csv("https://s3.amazonaws.com/hackerday.datascience/112/train.csv")
test_df = pd.read_csv("https://s3.amazonaws.com/hackerday.datascience/112/test.csv")
train_df.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity'],
      dtype='object', length=563)

In [3]:
train_df.Activity.value_counts()

WALKING               209
STANDING              179
LAYING                164
WALKING_UPSTAIRS      159
WALKING_DOWNSTAIRS    145
SITTING               143
Name: Activity, dtype: int64

In [4]:
X = train_df.iloc[:, :-1].values
y = train_df.iloc[:, -1].values

In [5]:
y[0:10]

array(['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING',
       'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING'],
      dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_y=LabelEncoder()
y=labelencoder_y.fit_transform(y)

In [7]:
y[0:100]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [8]:
y_o=pd.get_dummies(train_df.iloc[:, -1].values)

In [9]:
y_o[20:100]

,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
20,0,0,1,0,0,0
21,0,0,1,0,0,0
22,0,0,1,0,0,0
23,0,0,1,0,0,0
24,0,0,1,0,0,0
...,...,...,...,...,...,...
95,0,0,0,1,0,0
96,0,0,0,1,0,0
97,0,0,0,1,0,0
98,0,0,0,1,0,0


In [10]:
#splitting the training set in evaluation and train for our model validation 
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval=train_test_split(X, y, test_size= 0.2, random_state=0)

In [11]:
# Diving test set in test and validation
X_test=test_df.iloc[:,:-1].values
y_test=test_df.iloc[:,-1].values

In [12]:
y_test[0:10]

array(['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING',
       'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING'],
      dtype=object)

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_eval=LabelEncoder()
y_test=labelencoder_eval.fit_transform(y_test)

In [14]:
y_test[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Classfication using logistic regression

In [15]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=10)#complexity parameter
model

LogisticRegression(C=10)

In [16]:
model.fit(X_train, y_train)

C:\Users\Yash\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10)

In [17]:
# Now we will predict on splitted evaluation data from our training set
y_pred_train=model.predict(X_eval)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_eval, y_pred_train)

0.995

In [19]:
# Now we will test our model on test data
y_pred=model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8818818818818819

## Random forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
rf_pred=model.predict(X_eval)
accuracy_score(y_eval,rf_pred)

0.99

In [22]:
?RandomForestClassifier

In [23]:
# Now we will test our model on test data
y_pred=model.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9029029029029029

## Decision Tree

In [25]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'gini',max_depth=6,min_samples_leaf=2,random_state = 0)
classifier.fit(X_train, y_train)
df_pred=classifier.predict(X_eval)
accuracy_score(y_eval,df_pred)

0.935

In [26]:
?DecisionTreeClassifier

In [27]:
# Now we will test our model on test data
y_pred=classifier.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7787787787787788

## SVM

In [29]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier_svm = SVC(kernel = 'rbf', random_state = 0)
classifier_svm.fit(X_train, y_train)
svm_pred=classifier_svm.predict(X_eval)
accuracy_score(y_eval,svm_pred)

0.98

In [30]:
# Now we will test our model on test data
y_pred=classifier_svm.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8518518518518519

In [32]:
from sklearn.metrics import f1_score

In [33]:
f1_score(y_test, y_pred,average=None)

array([0.99450549, 0.68327402, 0.78947368, 0.87647059, 0.93536122,
       0.8253012 ])

### This gives F1 score for every class

## We are getting Random forest as best classfier algorithm on this dataset 

#### So to deploy the model we will save the model in form of pickle file , which later we will used for deployment

In [34]:
import pickle

In [35]:
pickle.dump(model, open('model.pkl','wb'))

In [36]:
# Loading model to compare the results
model = pickle.load( open('model.pkl','rb'))


In [37]:
test_df.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity'],
      dtype='object', length=563)

In [38]:
sample=test_df.loc[0,'tBodyAcc-mean()-X':'subject']

In [39]:
dict(sample)

{'tBodyAcc-mean()-X': 0.25717778,
 'tBodyAcc-mean()-Y': -0.02328523,
 'tBodyAcc-mean()-Z': -0.014653761999999999,
 'tBodyAcc-std()-X': -0.9384040000000001,
 'tBodyAcc-std()-Y': -0.92009078,
 'tBodyAcc-std()-Z': -0.66768331,
 'tBodyAcc-mad()-X': -0.95250112,
 'tBodyAcc-mad()-Y': -0.92524867,
 'tBodyAcc-mad()-Z': -0.67430222,
 'tBodyAcc-max()-X': -0.89408755,
 'tBodyAcc-max()-Y': -0.55457721,
 'tBodyAcc-max()-Z': -0.46622295,
 'tBodyAcc-min()-X': 0.71720847,
 'tBodyAcc-min()-Y': 0.6355024,
 'tBodyAcc-min()-Z': 0.78949666,
 'tBodyAcc-sma()': -0.87776423,
 'tBodyAcc-energy()-X': -0.99776606,
 'tBodyAcc-energy()-Y': -0.99841381,
 'tBodyAcc-energy()-Z': -0.93434525,
 'tBodyAcc-iqr()-X': -0.97566897,
 'tBodyAcc-iqr()-Y': -0.94982365,
 'tBodyAcc-iqr()-Z': -0.8304778,
 'tBodyAcc-entropy()-X': -0.16808416,
 'tBodyAcc-entropy()-Y': -0.37899553,
 'tBodyAcc-entropy()-Z': 0.24621698,
 'tBodyAcc-arCoeff()-X,1': 0.52120364,
 'tBodyAcc-arCoeff()-X,2': -0.48779311,
 'tBodyAcc-arCoeff()-X,3': 0.48228047,

In [40]:
output=test_df.loc[0,'Activity']
output

'STANDING'

## Now our model will give output as number as we have one hot encoded so how to map it to our labels I will take list of unique values and one hot encode it to see our model performance

In [41]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_o=LabelEncoder()

In [42]:
ot=train_df.Activity.unique()
ot

array(['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [43]:
ot_label=labelencoder_o.fit_transform(ot)

In [44]:
ot_label

array([2, 1, 0, 3, 4, 5])

In [45]:
print(model.predict([sample]))

[2]


## Now as we can see that our model is performing pretty good we can deploy it using flask

## Neural Networks

### Let us try to implement simple neural network and see how it performs on this dataset and does Deeplearing beats our traditional sklearn ML models

In [83]:
train_df = pd.read_csv("https://s3.amazonaws.com/hackerday.datascience/112/train.csv")
test_df = pd.read_csv("https://s3.amazonaws.com/hackerday.datascience/112/test.csv")

In [84]:
X = train_df.iloc[:, :-1].values
y = train_df.iloc[:, -1].values

In [85]:
# Diving test set 
X_test=test_df.iloc[:,:-1].values
y_test=test_df.iloc[:,-1].values

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [87]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_eval = sc.fit_transform(X_eval)
X_test = sc.transform(X_test)

In [88]:
y_train = pd.get_dummies(y_train)
y_eval =  pd.get_dummies(y_eval)

In [77]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
# Initialising the ANN
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 562, activation = 'tanh', input_dim = 562))

# Adding the second hidden layer
classifier.add(Dense(units = 128, activation = 'tanh'))
#classifier.add(Dense(units = 128, activation = 'tanh'))
classifier.add(Dropout(0.2))

classifier.add(Dense(units = 64, activation = 'tanh'))
#classifier.add(Dense(units = 64, activation = 'tanh'))
classifier.add(Dropout(0.2))

classifier.add(Dense(units = 32, activation = 'tanh'))
#classifier.add(Dense(units = 32, activation = 'tanh'))
classifier.add(Dropout(0.05))
# Adding the output layer
classifier.add(Dense(units = 6, activation = 'softmax'))
# Compiling the ANN
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# Fitting the ANN to the Training set
history = classifier.fit(X_train, y_train, 
                         validation_data = (X_eval, y_eval), 
                         batch_size = 28, 
                         epochs = 25)


Using TensorFlow backend.


Train on 899 samples, validate on 100 samples
Epoch 1/25
899/899 [==============================] - 3s 4ms/step - loss: 0.7979 - acc: 0.7041 - val_loss: 0.9979 - val_acc: 0.5100
Epoch 2/25
899/899 [==============================] - 1s 1ms/step - loss: 0.3908 - acc: 0.8765 - val_loss: 0.2823 - val_acc: 0.8700
Epoch 3/25
899/899 [==============================] - 1s 1ms/step - loss: 0.2143 - acc: 0.9199 - val_loss: 0.1356 - val_acc: 0.9500
Epoch 4/25
899/899 [==============================] - 1s 1ms/step - loss: 0.1402 - acc: 0.9655 - val_loss: 0.1924 - val_acc: 0.9500
Epoch 5/25
899/899 [==============================] - 1s 1ms/step - loss: 0.1084 - acc: 0.9700 - val_loss: 0.0943 - val_acc: 0.9700
Epoch 6/25
899/899 [==============================] - 1s 994us/step - loss: 0.0663 - acc: 0.9844 - val_loss: 0.1273 - val_acc: 0.9600
Epoch 7/25
899/899 [==============================] - 1s 1ms/step - loss: 0.0579 - acc: 0.9855 - val_loss: 0.0574 - val_acc: 0.9800
Epoch 8/25
899/899 [========

In [78]:
# Prediction
test_pred = classifier.predict(X_test)

In [79]:
# Mark probability score > 0.5 as Predicted Label, axis = 1 means insert column-wise 
results = test_pred.argmax(axis=1)

In [80]:
results[:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [81]:
labelencoder_y=LabelEncoder()
y_test=labelencoder_y.fit_transform(y_test)

In [82]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, results)

0.8478478478478478

### We can see our sklearn gives better accuracy with less efforts